In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark import SparkConf

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df=spark.read.format("csv").option("header","true").option("inferSchema",'True').load('2018.csv')

In [3]:
#df.createOrReplaceTempView("df1")

In [4]:
#df=spark.sql('select * from df1')
#df.show()

In [2]:
df_pandas = df.limit(5).toPandas()
display(df_pandas)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,None
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,None
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,None
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,None
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,None


In [3]:
df.count()

7213446

In [4]:
df.distinct().count()
#This shows there are no duplicate rows

7213446

In [8]:
df.printSchema()

root
 |-- FL_DATE: timestamp (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY:

In [5]:
c=df.count()
for i in df.columns:
    print(i,(20-len(i))*"-",((df.filter(df[i].isNull()).count())/c)*100)

FL_DATE ------------- 0.0
OP_CARRIER ---------- 0.0
OP_CARRIER_FL_NUM --- 0.0
ORIGIN -------------- 0.0
DEST ---------------- 0.0
CRS_DEP_TIME -------- 0.0
DEP_TIME ------------ 1.5570505414471807
DEP_DELAY ----------- 1.6252149111534209
TAXI_OUT ------------ 1.6057512595228411
WHEELS_OFF ---------- 1.605737396523104
WHEELS_ON ----------- 1.653107266624024
TAXI_IN ------------- 1.653107266624024
CRS_ARR_TIME -------- 0.0
ARR_TIME ------------ 1.653093403624287
ARR_DELAY ----------- 1.8997854839420716
CANCELLED ----------- 0.0
CANCELLATION_CODE --- 98.38379603867556
DIVERTED ------------ 0.0
CRS_ELAPSED_TIME ---- 0.00013862999736880265
ACTUAL_ELAPSED_TIME - 1.8637694106256566
AIR_TIME ------------ 1.8637694106256566
DISTANCE ------------ 0.0
CARRIER_DELAY ------- 81.2473816259247
WEATHER_DELAY ------- 81.2473816259247
NAS_DELAY ----------- 81.2473816259247
SECURITY_DELAY ------ 81.2473816259247
LATE_AIRCRAFT_DELAY - 81.2473816259247
Unnamed: 27 --------- 100.0


In [6]:
df1=df

In [7]:
c=df.count()
for i in df.columns:
    if (((df.filter(df[i].isNull()).count())/c)*100)>80:
        df=df.drop(i)
df.show()    

+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|
+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|2018-01-01 00:00:00|        UA|             2429|   EWR| DEN|        1517|  1512.0|     -5.0|    15.0|    1527.0|   1712.0|   10.0|        1745|  1722.0|    -23.0|      0.0|     0.0|           268.0|              250.0|   225.0|  1605.0|
|2018-01-01 00:00:00|        UA|            

In [8]:
c=df.count()
for i in df.columns:
    print(i,(20-len(i))*"-",((df.filter(df[i].isNull()).count())/c)*100)

FL_DATE ------------- 0.0
OP_CARRIER ---------- 0.0
OP_CARRIER_FL_NUM --- 0.0
ORIGIN -------------- 0.0
DEST ---------------- 0.0
CRS_DEP_TIME -------- 0.0
DEP_TIME ------------ 1.5570505414471807
DEP_DELAY ----------- 1.6252149111534209
TAXI_OUT ------------ 1.6057512595228411
WHEELS_OFF ---------- 1.605737396523104
WHEELS_ON ----------- 1.653107266624024
TAXI_IN ------------- 1.653107266624024
CRS_ARR_TIME -------- 0.0
ARR_TIME ------------ 1.653093403624287
ARR_DELAY ----------- 1.8997854839420716
CANCELLED ----------- 0.0
DIVERTED ------------ 0.0
CRS_ELAPSED_TIME ---- 0.00013862999736880265
ACTUAL_ELAPSED_TIME - 1.8637694106256566
AIR_TIME ------------ 1.8637694106256566
DISTANCE ------------ 0.0


In [9]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col

df=df.withColumn("FL_DATE",to_date(col("FL_DATE"),"yyyy-MM-dd"))

In [10]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[FL_DATE: date, OP_CARRIER: string, OP_CARRIER_FL_NUM: int, ORIGIN: string, DEST: string, CRS_DEP_TIME: int, DEP_TIME: double, DEP_DELAY: double, TAXI_OUT: double, WHEELS_OFF: double, WHEELS_ON: double, TAXI_IN: double, CRS_ARR_TIME: int, ARR_TIME: double, ARR_DELAY: double, CANCELLED: double, DIVERTED: double, CRS_ELAPSED_TIME: double, ACTUAL_ELAPSED_TIME: double, AIR_TIME: double, DISTANCE: double]>

In [11]:
from pyspark.sql.types import StringType
df = df.withColumn("OP_CARRIER_FL_NUM",df["OP_CARRIER_FL_NUM"].cast(StringType()))

In [12]:
df.select('OP_CARRIER_FL_NUM').dtypes

[('OP_CARRIER_FL_NUM', 'string')]

In [13]:
df.select('OP_CARRIER').distinct().show()

+----------+
|OP_CARRIER|
+----------+
|        UA|
|        NK|
|        AA|
|        EV|
|        B6|
|        DL|
|        OO|
|        F9|
|        YV|
|        MQ|
|        OH|
|        HA|
|        G4|
|        YX|
|        AS|
|        VX|
|        WN|
|        9E|
+----------+



In [14]:
from pyspark.sql.functions import regexp_replace
l={'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines'}
l=list(l.items())
for i in range(18):
    df=df.withColumn('OP_CARRIER', regexp_replace('OP_CARRIER', l[i][0], l[i][1]))


In [15]:
df.select('OP_CARRIER').distinct().show()

+------------------+
|        OP_CARRIER|
+------------------+
|      Mesa Airline|
|     Allegiant Air|
|   United Airlines|
|    Virgin America|
| Hawaiian Airlines|
|  Republic Airways|
|        ExpressJet|
|  SkyWest Airlines|
| Frontier Airlines|
|      Endeavor Air|
| American Airlines|
|   JetBlue Airways|
|         Envoy Air|
|    Delta Airlines|
|      PSA Airlines|
|   Alaska Airlines|
|   Spirit Airlines|
|Southwest Airlines|
+------------------+



In [16]:
df.agg({'CANCELLED': 'sum'}).show()

+--------------+
|sum(CANCELLED)|
+--------------+
|      116584.0|
+--------------+



In [21]:
df.select('CANCELLED').distinct().show()

+---------+
|CANCELLED|
+---------+
|      0.0|
|      1.0|
+---------+



In [17]:
canceled=df.filter(df['CANCELLED']>0)

In [18]:
canceled.show()

+----------+---------------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|   FL_DATE|     OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|
+----------+---------------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|2018-01-01|United Airlines|             2034|   IAH| MFE|        1440|    null|     null|    null|      null|     null|   null|        1601|    null|     null|      1.0|     0.0|            81.0|               null|    null|   316.0|
|2018-01-01|United Airlines|              864|   LAS| SFO|  

In [20]:
df_new=df.filter(df['ARR_DELAY']>0)
df_new.show() #df_new contains delayed flights only

+----------+---------------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|   FL_DATE|     OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|
+----------+---------------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|2018-01-01|United Airlines|             2424|   ORD| ALB|         630|   650.0|     20.0|    13.0|     703.0|    926.0|   10.0|         922|   936.0|     14.0|      0.0|     0.0|           112.0|              106.0|    83.0|   723.0|
|2018-01-01|United Airlines|             2416|   ORD| CLE|  

In [21]:
df=df.withColumn('ARR_DELAY',col('ARR_DELAY')/60)
df=df.withColumn('DEP_DELAY',col('DEP_DELAY')/60)

In [22]:
from pyspark.sql.functions import year
from pyspark.sql.functions import month
from pyspark.sql.functions import dayofweek
df_new=df_new.withColumn('FL_YEAR',year(df_new.FL_DATE))
df_new=df_new.withColumn('FL_MONTH',month(df_new.FL_DATE))
df_new=df_new.withColumn('FL_DAYOFWEEK',dayofweek(df_new.FL_DATE))

In [23]:
bd1 = df.withColumn('Delayed', (df.ARR_DELAY >=15).cast('int'))

In [24]:
bd1.select('Delayed').distinct().show()

+-------+
|Delayed|
+-------+
|   null|
|      1|
|      0|
+-------+



In [25]:
bd1.createOrReplaceTempView("bd1")

bd1 = spark.sql("select *, case \
               when DEP_TIME <= 800 then 1 \
               when 800 < DEP_TIME and DEP_TIME <= 1200 then 2 \
               when 1200 < DEP_TIME and DEP_TIME <= 1600 then 3 \
               when 1600 < DEP_TIME and DEP_TIME <= 2100 then 4 \
               else 1 end as TimeSlot \
               from bd1")

In [26]:
from pyspark.ml.feature import StringIndexer


indexer1 = StringIndexer(inputCol='OP_CARRIER',outputCol='INDEX_CARRIER') 
bd2=indexer1.fit(bd1).transform(bd1)

indexer2 = StringIndexer(inputCol='ORIGIN',outputCol='INDEX_ORIGIN') 
bd3=indexer2.fit(bd2).transform(bd2)

bd3.groupBy('OP_CARRIER','INDEX_CARRIER').count().sort('INDEX_CARRIER').show()

+------------------+-------------+-------+
|        OP_CARRIER|INDEX_CARRIER|  count|
+------------------+-------------+-------+
|Southwest Airlines|          0.0|1352552|
|    Delta Airlines|          1.0| 949283|
| American Airlines|          2.0| 916818|
|  SkyWest Airlines|          3.0| 774137|
|   United Airlines|          4.0| 621565|
|  Republic Airways|          5.0| 316090|
|   JetBlue Airways|          6.0| 305010|
|         Envoy Air|          7.0| 296001|
|      PSA Airlines|          8.0| 278457|
|      Endeavor Air|          9.0| 245917|
|   Alaska Airlines|         10.0| 245761|
|      Mesa Airline|         11.0| 215138|
|        ExpressJet|         12.0| 202890|
|   Spirit Airlines|         13.0| 176178|
| Frontier Airlines|         14.0| 120035|
|     Allegiant Air|         15.0|  96221|
| Hawaiian Airlines|         16.0|  83723|
|    Virgin America|         17.0|  17670|
+------------------+-------------+-------+



In [28]:
from pyspark.sql.functions import *

df.stat.corr('FL_DAYOFWEEK','ARR_DELAY')

AnalysisException: Cannot resolve column name "FL_DAYOFWEEK" among (FL_DATE, OP_CARRIER, OP_CARRIER_FL_NUM, ORIGIN, DEST, CRS_DEP_TIME, DEP_TIME, DEP_DELAY, TAXI_OUT, WHEELS_OFF, WHEELS_ON, TAXI_IN, CRS_ARR_TIME, ARR_TIME, ARR_DELAY, CANCELLED, DIVERTED, CRS_ELAPSED_TIME, ACTUAL_ELAPSED_TIME, AIR_TIME, DISTANCE)

In [29]:
bd4=bd3.select('DEP_DELAY',
             'DISTANCE', 
             'FL_DAYOFWEEK',
             'INDEX_CARRIER', 
             'TimeSlot',
             'Delayed',
             'FL_MONTH',
             'ACTUAL_ELAPSED_TIME',
             'INDEX_ORIGIN')
             
bd4.limit(10).toPandas()

AnalysisException: Column 'FL_DAYOFWEEK' does not exist. Did you mean one of the following? [bd1.FL_DATE, bd1.AIR_TIME, bd1.ARR_DELAY, bd1.ARR_TIME, bd1.CANCELLED, bd1.DEP_TIME, bd1.DEST, bd1.DISTANCE, bd1.DIVERTED, INDEX_ORIGIN, bd1.ORIGIN, bd1.DEP_DELAY, bd1.Delayed, INDEX_CARRIER, bd1.TAXI_IN, bd1.TAXI_OUT, TimeSlot, bd1.OP_CARRIER, bd1.WHEELS_ON, bd1.CRS_DEP_TIME, bd1.WHEELS_OFF, bd1.CRS_ARR_TIME, bd1.CRS_ELAPSED_TIME, bd1.OP_CARRIER_FL_NUM, bd1.ACTUAL_ELAPSED_TIME];
'Project [DEP_DELAY#3836, DISTANCE#38, 'FL_DAYOFWEEK, INDEX_CARRIER#4043, TimeSlot#3960, Delayed#3930, 'FL_MONTH, ACTUAL_ELAPSED_TIME#36, INDEX_ORIGIN#4129]
+- Project [FL_DATE#3007, OP_CARRIER#3433, OP_CARRIER_FL_NUM#3029, ORIGIN#20, DEST#21, CRS_DEP_TIME#22, DEP_TIME#23, DEP_DELAY#3836, TAXI_OUT#25, WHEELS_OFF#26, WHEELS_ON#27, TAXI_IN#28, CRS_ARR_TIME#29, ARR_TIME#30, ARR_DELAY#3814, CANCELLED#32, DIVERTED#34, CRS_ELAPSED_TIME#35, ACTUAL_ELAPSED_TIME#36, AIR_TIME#37, DISTANCE#38, Delayed#3930, TimeSlot#3960, INDEX_C